In [1]:
using Pkg
Pkg.add("PyCall")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using CSV
using DataFrames
using Plots
using StatsPlots
using Statistics

In [3]:
#file_path = "/ibmm_data2/oas_database/paired_lea_tmp/paired_model/peak_analysis/bert2gpt/data/global_align.csv"

file_path = "/ibmm_data2/oas_database/paired_lea_tmp/paired_model/BERT2BERT/embedding_analysis/data/bert2gpt_df_merged_final_test_set_with_full_paired.csv"

df = CSV.read(file_path, DataFrame);
names(df)

47-element Vector{String}:
 "sequence_heavy"
 "locus_heavy"
 "v_call_heavy_x"
 "sequence_alignment_heavy"
 "sequence_alignment_aa_heavy_x"
 "germline_alignment_aa_heavy"
 "cdr3_aa_heavy"
 "sequence_light"
 "locus_light_x"
 "v_call_light_x"
 ⋮
 "locus_light_y"
 "v_call_heavy_y"
 "v_call_light_y"
 "j_call_heavy"
 "j_call_light"
 "v_gene_heavy_family_y"
 "v_gene_light_family_y"
 "j_gene_heavy_family"
 "j_gene_light_family"

In [4]:
using DataFrames, PyCall

# Import Python modules
sns = pyimport("seaborn")
plt = pyimport("matplotlib.pyplot")
pd = pyimport("pandas")

# Assume your Julia DataFrame is named `df`
py_df = pd.DataFrame(Dict(col => df[!, col] for col in names(df)))

PyObject Ptr{PyCall.PyObject_struct} @0x00007f94cd0294d0

In [5]:
# Start with a fresh figure to avoid overlapping previous plots
plt.figure()

bins = 40
sns.histplot(data=py_df, x="calculated_similarity", bins=bins, kde=true, 
             color="blue", edgecolor="white", alpha=0.3)

plt.xlabel("Similarity betw. generated and true light sequence")
plt.ylabel("Frequency")

# Save before showing the plot
plt.savefig("/ibmm_data2/oas_database/paired_lea_tmp/paired_model/peak_analysis/bert2gpt/plots/control/final_blue_bins_40_poly_kde_orange.png", dpi=300)
plt.show()

In [23]:
extracted_data_merged = CSV.read("/ibmm_data2/oas_database/paired_lea_tmp/paired_model/peak_analysis/bert2bert/data/extracted_data_merged_header.csv", DataFrame)


Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String31,String7,String7,String15,String31,String15
1,GGGAGGCTTTCTAAGAGTCATGGATCTCATGTGCAAGAAAATGAAGCACCTGTGGTTCTTCCTCCTGCTGGTGGCGGCTCCCAGATGGATCGTGTCCCAGTTGCAGGTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACGTGCACTGTCTCTGGTGCCTCCAGTAGTATTAAAAAATACTACTGGGGCTGGATCCGCCAGTCCCCCGGGAAGGGTCTGGAGTGGATTGGGAGTATCTATTCTAGCGGGAGTACGCAGTACAATCCGGCCCTCGGGAGTCGAGTCACCCTCTCCGTGGACACGTCCCAGACCCAGTTCTCCCTGAGGCTGACCTCTGTGACCGCCGCAGACACGGCTACGTACTTCTGTGCGAGACAGGGGGCGGATTGTACTGACGGCAGCTGCTACCTCAATGATGCTTTTGATGTTTGGGGCCGCGGGACAGTGGTCACCGTCTCTTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCGACAGCACCCCCCAAGATGGGAACGTGGTCGTCGCATGCCTGGTCCAGGGCTTCTTCCCCCAGGAGCCACTCAGTGTGACCTGGAGCGAAAGCGGACAGAACGTGACCGCCAGAAACTTCCC,H,IGHV4-39*01,CAGTTGCAGGTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACGTGCACTGTCTCTGGTGCCTCCAGTAGTATTAAAAAATACTACTGGGGCTGGATCCGCCAGTCCCCCGGGAAGGGTCTGGAGTGGATTGGGAGTATCTATTCTAGCGGGAGTACGCAGTACAATCCGGCCCTCGGGAGTCGAGTCACCCTCTCCGTGGACACGTCCCAGACCCAGTTCTCCCTGAGGCTGACCTCTGTGACCGCCGCAGACACGGCTACGTACTTCTGTGCGAGACAGGGGGCGGATTGTACTGACGGCAGCTGCTACCTCAATGATGCTTTTGATGTTTGGGGCCGCGGGACAGTGGTCACCGTCTCTTCAG,QLQVQESGPGLVKPSETLSLTCTVSGASSSIKKYYWGWIRQSPGKGLEWIGSIYSSGSTQYNPALGSRVTLSVDTSQTQFSLRLTSVTAADTATYFCARQGADCTDGSCYLNDAFDVWGRGTVVTVSS,QLQLQESGPGLVKPSETLSLTCTVSGGSISSSSYYWGWIRQPPGKGLEWIGSIYYSGSTYYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARXXXXCSGGSCYXXDAFDVWGQGTMVTVSS,ARQGADCTDGSCYLNDAFDV,AGGAGTCAGACCCAGTCAGGACACAGCATGGACATGAGGGTCCCCGCTCAGCTCCTGGGGCTCCTGCTGCTCTGGCTCCCAGGTGCCAAATGTGACATCCAGATGACCCAGTCTCCTTCCACCTTGTCTGCATCTGTAGGAGACAGAGTCACCATCACTTGCCGGGCCAGTCACAGTATTAATACTTGGTTGGCCTGGTATCAGCAGAAACCAGGGAAAGCCCCTAAACTCCTGATATATAAGGCGTCTTCTTTAGAAAAGGGGGTCCCATCAAGGTTCAGCGGCAGTGGATCTGGGACCGAATTCACTCTCACCATCAACAGCCTGCAGCCTGATGATTTTGCAACTTATTACTGCCAACACTATAATAGTTATCCATTCGCTTTTGGCCAGGGGACCATGCTGGAGATCGAACGAACTGTGGCTGCACCATCTGTCTTCATCTTCCCGCCATCTGATGAGCAGTTGAAATCTGGAACTGCCTCTGTTGTGTGCCTGCTGAATAACTTCTATCCCAGAGAGGCCAAAGTACAGTGGAAGGTGGATAACGC,K,IGKV1-5*03,GACATCCAGATGACCCAGTCTCCTTCCACCTTGTCTGCATCTGTAGGAGACAGAGTCACCATCACTTGCCGGGCCAGTCACAGTATTAATACTTGGTTGGCCTGGTATCAGCAGAAACCAGGGAAAGCCCCTAAACTCCTGATATATAAGGCGTCTTCTTTAGAAAAGGGGGTCCCATCAAGGTTCAGCGGCAGTGGATCTGGGACCGAATTCACTCTCACCATCAACAGCCTGCAGCCTGATGATTTTGCAACTTATTACTGCCAACACTATAATAGTTATCCATTCGCTTTTGGCCAGGGGACCATGCTGGAGATCGAAC,DIQMTQSPSTLSASVGDRVTITCRASHSINTWLAWYQQKPGKAPKLLIYKASSLEKGVPSRFSGSGSGTEFTLTINSLQPDDFATYYCQHYNSYPFAFGQGTMLEIE,DIQMTQSPSTLSASVGDRVTITCRASQSISSWLAWYQQKPGKAPKLLIYKASSLESGVPSRFSGSGSGTEFTLTISSLQPDDFATYYCQQYNSYXXXFGQGTKLEIK,QHYNSYPFA,QLQVQESGPGLVKPSETLSLTCTVSGASSSIKKYYWGWIRQSPGKGLEWIGSIYSSGSTQYNPALGSRVTLSVDTSQTQFSLRLTSVTAADTATYFCARQGADCTDGSCYLNDAFDVWGRGTVVTVSS[SEP]DIQMTQSPSTLSASVGDRVTITCRASHSINTWLAWYQQKPGKAPKLLIYKASSLEKGVPSRFSGSGSGTEFTLTINSLQPDDFATYYCQHYNSYPFAFGQGTMLEIE,Unsorted-B-Cells,None,human,390c,"James et al, 2020",65 to 70
2,TTTCTTATATGGGGATGCTTTCTGAGAGTCATGGACCTCCTGTGCAAGAACATGAAGCACCTGTGGTTCTTCCTCCTGCTGGTGGCGGCTCCCAGATGGGTCCTGTCCCAGTTGCAGTTGCAGGAGTCGGGCCCGGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCATCTGCAGTGTCTCTGGTGGCTCCATCACCACTAGCAGCTACTACTGGGCCTGGATCCGCCAGTCCCCAGGGAAGGGTCTGGAGTGGATTGGGACTATGTATAATACTGGGAGCACCTACTACAACCCGTCACTCAAGAGTCGGGTCACCATGTCAGTTCACACGTCCGAGAACCGCTTCTCCCTGGACCTGACGTCTGTGACCGCCGCTGACGCGGCCGTCTATTACTGTGCGAGAGAGGAATTGGGAAACACTTCGTCTGCTTACTTTTTTGACTACTGGGGCCCGGGGGCCCTGGTCACCGTCTCCTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCGACAGCACCCCCCAAGATGGGAACGTGGTCGTCGCATGCCTGGTCCAGGGCTTCTTCCCCCAGGAGCCACTCAGTGTGACCTGGAGCGAAAGCGGACAGAACGTGACCGCCAGAAACTTCCC,H,IGHV4-39*07,CAGTTGCAGTTGCAGGAGTCGGGCCCGGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCATCTGCAGTGTCTCTGGTGGCTCCA

In [7]:
# Assuming `df` is your DataFrame
df[!, :heavy_sep_light] = string.(df.input_heavy_sequence,"[SEP]", df.sequence_alignment_aa_light,)

# Display the first few rows to verify
println(first(df, 10))

10×9 DataFrame
 Row │ sequence_alignment_aa_light        generated_sequence_light           input_heavy_sequence               BLOSUM_score  similarity  perplexity  calculated_blosum  calculated_similarity  heavy_sep_light                   
     │ String                             String                             String                             Int64         Float64     Float64     Int64              Float64                String                            
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ DIQMTQSPSTLSASVGDRVTITCRASHSINTW…  DIVMTQSPDSLAVSLGERATINCKSSQSVLYS…  QLQVQESGPGLVKPSETLSLTCTVSGASSSIK…           349     63.7168     2.39702                345                63.7168  QLQVQESGPGLVKPSETLSLTCTVSGASSSIK…
   2 │ QSALTQPASVSGSPGQSITISCSGTSDDIGDY…  DIQMTQSPSSLSASVGDRVTITCRASQSISTY…  Q

In [8]:
df_merged_final = innerjoin(extracted_data_merged, df, on=(:sequence_alignment_heavy_sep_light => :heavy_sep_light), makeunique=true)

Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age,sequence_alignment_aa_light_1,generated_sequence_light,input_heavy_sequence,BLOSUM_score,similarity,perplexity,calculated_blosum,calculated_similarity
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String31,String7,String7,String15,String31,String15,String,String,String,Int64,Float64,Float64,Int64,Float64
1,GGGAGGCTTTCTAAGAGTCATGGATCTCATGTGCAAGAAAATGAAGCACCTGTGGTTCTTCCTCCTGCTGGTGGCGGCTCCCAGATGGATCGTGTCCCAGTTGCAGGTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACGTGCACTGTCTCTGGTGCCTCCAGTAGTATTAAAAAATACTACTGGGGCTGGATCCGCCAGTCCCCCGGGAAGGGTCTGGAGTGGATTGGGAGTATCTATTCTAGCGGGAGTACGCAGTACAATCCGGCCCTCGGGAGTCGAGTCACCCTCTCCGTGGACACGTCCCAGACCCAGTTCTCCCTGAGGCTGACCTCTGTGACCGCCGCAGACACGGCTACGTACTTCTGTGCGAGACAGGGGGCGGATTGTACTGACGGCAGCTGCTACCTCAATGATGCTTTTGATGTTTGGGGCCGCGGGACAGTGGTCACCGTCTCTTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCGACAGCACCCCCCAAGATGGGAACGTGGTCGTCGCATGCCTGGTCCAGGGCTTCTTCCCCCAGGAGCCACTCAGTGTGACCTGGAGCGAAAGCGGACAGAACGTGACCGCCAGAAACTTCCC,H,IGHV4-39*01,CAGTTGCAGGTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACGTGCACTGTCTCTGGTGCCTCCAGTAGTATTAAAAAATACTACTGGGGCTGGATCCGCCAGTCCCCCGGGAAGGGTCTGGAGTGGATTGGGAGTATCTATTCTAGCGGGAGTACGCAGTACAATCCGGCCCTCGGGAGTCGAGTCACCCTCTCCGTGGACACGTCCCAGACCCAGTTCTCCCTGAGGCTGACCTCTGTGACCGCCGCAGACACGGCTACGTACTTCTGTGCGAGACAGGGGGCGGATTGTACTGACGGCAGCTGCTACCTCAATGATGCTTTTGATGTTTGGGGCCGCGGGACAGTGGTCACCGTCTCTTCAG,QLQVQESGPGLVKPSETLSLTCTVSGASSSIKKYYWGWIRQSPGKGLEWIGSIYSSGSTQYNPALGSRVTLSVDTSQTQFSLRLTSVTAADTATYFCARQGADCTDGSCYLNDAFDVWGRGTVVTVSS,QLQLQESGPGLVKPSETLSLTCTVSGGSISSSSYYWGWIRQPPGKGLEWIGSIYYSGSTYYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARXXXXCSGGSCYXXDAFDVWGQGTMVTVSS,ARQGADCTDGSCYLNDAFDV,AGGAGTCAGACCCAGTCAGGACACAGCATGGACATGAGGGTCCCCGCTCAGCTCCTGGGGCTCCTGCTGCTCTGGCTCCCAGGTGCCAAATGTGACATCCAGATGACCCAGTCTCCTTCCACCTTGTCTGCATCTGTAGGAGACAGAGTCACCATCACTTGCCGGGCCAGTCACAGTATTAATACTTGGTTGGCCTGGTATCAGCAGAAACCAGGGAAAGCCCCTAAACTCCTGATATATAAGGCGTCTTCTTTAGAAAAGGGGGTCCCATCAAGGTTCAGCGGCAGTGGATCTGGGACCGAATTCACTCTCACCATCAACAGCCTGCAGCCTGATGATTTTGCAACTTATTACTGCCAACACTATAATAGTTATCCATTCGCTTTTGGCCAGGGGACCATGCTGGAGATCGAACGAACTGTGGCTGCACCATCTGTCTTCATCTTCCCGCCATCTGATGAGCAGTTGAAATCTGGAACTGCCTCTGTTGTGTGCCTGCTGAATAACTTCTATCCCAGAGAGGCCAAAGTACAGTGGAAGGTGGATAACGC,K,IGKV1-5*03,GACATCCAGATGACCCAGTCTCCTTCCACCTTGTCTGCATCTGTAGGAGACAGAGTCACCATCACTTGCCGGGCCAGTCACAGTATTAATACTTGGTTGGCCTGGTATCAGCAGAAACCAGGGAAAGCCCCTAAACTCCTGATATATAAGGCGTCTTCTTTAGAAAAGGGGGTCCCATCAAGGTTCAGCGGCAGTGGATCTGGGACCGAATTCACTCTCACCATCAACAGCCTGCAGCCTGATGATTTTGCAACTTATTACTGCCAACACTATAATAGTTATCCATTCGCTTTTGGCCAGGGGACCATGCTGGAGATCGAAC,DIQMTQSPSTLSASVGDRVTITCRASHSINTWLAWYQQKPGKAPKLLIYKASSLEKGVPSRFSGSGSGTEFTLTINSLQPDDFATYYCQHYNSYPFAFGQGTMLEIE,DIQMTQSPSTLSASVGDRVTITCRASQSISSWLAWYQQKPGKAPKLLIYKASSLESGVPSRFSGSGSGTEFTLTISSLQPDDFATYYCQQYNSYXXXFGQGTKLEIK,QHYNSYPFA,QLQVQESGPGLVKPSETLSLTCTVSGASSSIKKYYWGWIRQSPGKGLEWIGSIYSSGSTQYNPALGSRVTLSVDTSQTQFSLRLTSVTAADTATYFCARQGADCTDGSCYLNDAFDVWGRGTVVTVSS[SEP]DIQMTQSPSTLSASVGDRVTITCRASHSINTWLAWYQQKPGKAPKLLIYKASSLEKGVPSRFSGSGSGTEFTLTINSLQPDDFATYYCQHYNSYPFAFGQGTMLEIE,Unsorted-B-Cells,None,human,390c,"James et al, 2020",65 to 70,DIQMTQSPSTLSASVGDRVTITCRASHSINTWLAWYQQKPGKAPKLLIYKASSLEKGVPSRFSGSGSGTEFTLTINSLQPDDFATYYCQHYNSYPFAFGQGTMLEIE,DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNNKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSSPQTFGQGTKLEIK,QLQVQESGPGLVKPSETLSLTCTVSGASSSIKKYYWGWIRQSPGKGLEWIGSIYSSGSTQYNPALGSRVTLSVDTSQTQFSLRLTSVTAADTATYFCARQGADCTDGSCYLNDAFDVWGRGTVVTVSS,349,63.7168,2.39702,345,63.7168
2,TGGGAGGCTTTCTAAGAGTCATGGATCTCATGTGCAAGAAAATGAAGCACCTGTGGTTCTTCCTCCTGCTGGTGGCGGCTCCCAGATGGATCGTGTCCCAGTTGCAGGTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCC

In [24]:
# Extract up to the first two segments (e.g., "IGKV2-30" from "IGKV2-30*01")
df_merged_final[!, :v_gene_heavy_family] = replace.(df_merged_final.v_call_heavy, r"(^[^*]+?)(?:\-.*)?$" => s"\1"); # the regex removes any part of the string after (and including) the first *, keeping only the portion before it.
df_merged_final[!, :v_gene_light_family] = replace.(df_merged_final.v_call_light, r"(^[^*]+?)(?:\-.*)?$" => s"\1");

In [25]:
using DataFrames

# Filter out rows where `locus_light` equals "H"
df_merged_final_filtered = filter(row -> row.locus_light != "H", df_merged_final)

# Convert the locus_light column to standard Strings
df_merged_final_filtered.locus_light = string.(df_merged_final_filtered.locus_light)


52136-element Vector{String1}:
 "K"
 "K"
 "K"
 "K"
 "L"
 "L"
 "L"
 "L"
 "K"
 "K"
 ⋮
 "K"
 "K"
 "K"
 "K"
 "K"
 "K"
 "L"
 "K"
 "K"

In [26]:
# if there is a K in the locus_light column, rename it to Kappa, rename it to Lambda otherwise
df_merged_final_filtered.locus_light = ifelse.(occursin.("K", df_merged_final_filtered.locus_light), "Kappa", "Lambda")

52136-element Vector{String}:
 "Kappa"
 "Kappa"
 "Kappa"
 "Kappa"
 "Lambda"
 "Lambda"
 "Lambda"
 "Lambda"
 "Kappa"
 "Kappa"
 ⋮
 "Kappa"
 "Kappa"
 "Kappa"
 "Kappa"
 "Kappa"
 "Kappa"
 "Lambda"
 "Kappa"
 "Kappa"

In [27]:
# remove all rows which have L in the column 
df_merged_final_filtered = filter(row -> row.locus_light != "Lambda", df_merged_final_filtered)

# remove all rows which have D in the string in the column v_gene_light_family
#df_merged_final_filtered = filter(row -> !occursin("D", row.v_gene_light_family), df_merged_final_filtered)

Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age,sequence_alignment_aa_light_1,generated_sequence_light,input_heavy_sequence,BLOSUM_score,similarity,perplexity,calculated_blosum,calculated_similarity,v_gene_heavy_family,v_gene_light_family
,String,String1,String15,String,String,String,String,String,String,String15,String,String,String,String31,String,String31,String7,String7,String15,String31,String15,String,String,String,Int64,Float64,Float64,Int64,Float64,String,String
1,GGGAGGCTTTCTAAGAGTCATGGATCTCATGTGCAAGAAAATGAAGCACCTGTGGTTCTTCCTCCTGCTGGTGGCGGCTCCCAGATGGATCGTGTCCCAGTTGCAGGTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACGTGCACTGTCTCTGGTGCCTCCAGTAGTATTAAAAAATACTACTGGGGCTGGATCCGCCAGTCCCCCGGGAAGGGTCTGGAGTGGATTGGGAGTATCTATTCTAGCGGGAGTACGCAGTACAATCCGGCCCTCGGGAGTCGAGTCACCCTCTCCGTGGACACGTCCCAGACCCAGTTCTCCCTGAGGCTGACCTCTGTGACCGCCGCAGACACGGCTACGTACTTCTGTGCGAGACAGGGGGCGGATTGTACTGACGGCAGCTGCTACCTCAATGATGCTTTTGATGTTTGGGGCCGCGGGACAGTGGTCACCGTCTCTTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCGACAGCACCCCCCAAGATGGGAACGTGGTCGTCGCATGCCTGGTCCAGGGCTTCTTCCCCCAGGAGCCACTCAGTGTGACCTGGAGCGAAAGCGGACAGAACGTGACCGCCAGAAACTTCCC,H,IGHV4-39*01,CAGTTGCAGGTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACGTGCACTGTCTCTGGTGCCTCCAGTAGTATTAAAAAATACTACTGGGGCTGGATCCGCCAGTCCCCCGGGAAGGGTCTGGAGTGGATTGGGAGTATCTATTCTAGCGGGAGTACGCAGTACAATCCGGCCCTCGGGAGTCGAGTCACCCTCTCCGTGGACACGTCCCAGACCCAGTTCTCCCTGAGGCTGACCTCTGTGACCGCCGCAGACACGGCTACGTACTTCTGTGCGAGACAGGGGGCGGATTGTACTGACGGCAGCTGCTACCTCAATGATGCTTTTGATGTTTGGGGCCGCGGGACAGTGGTCACCGTCTCTTCAG,QLQVQESGPGLVKPSETLSLTCTVSGASSSIKKYYWGWIRQSPGKGLEWIGSIYSSGSTQYNPALGSRVTLSVDTSQTQFSLRLTSVTAADTATYFCARQGADCTDGSCYLNDAFDVWGRGTVVTVSS,QLQLQESGPGLVKPSETLSLTCTVSGGSISSSSYYWGWIRQPPGKGLEWIGSIYYSGSTYYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARXXXXCSGGSCYXXDAFDVWGQGTMVTVSS,ARQGADCTDGSCYLNDAFDV,AGGAGTCAGACCCAGTCAGGACACAGCATGGACATGAGGGTCCCCGCTCAGCTCCTGGGGCTCCTGCTGCTCTGGCTCCCAGGTGCCAAATGTGACATCCAGATGACCCAGTCTCCTTCCACCTTGTCTGCATCTGTAGGAGACAGAGTCACCATCACTTGCCGGGCCAGTCACAGTATTAATACTTGGTTGGCCTGGTATCAGCAGAAACCAGGGAAAGCCCCTAAACTCCTGATATATAAGGCGTCTTCTTTAGAAAAGGGGGTCCCATCAAGGTTCAGCGGCAGTGGATCTGGGACCGAATTCACTCTCACCATCAACAGCCTGCAGCCTGATGATTTTGCAACTTATTACTGCCAACACTATAATAGTTATCCATTCGCTTTTGGCCAGGGGACCATGCTGGAGATCGAACGAACTGTGGCTGCACCATCTGTCTTCATCTTCCCGCCATCTGATGAGCAGTTGAAATCTGGAACTGCCTCTGTTGTGTGCCTGCTGAATAACTTCTATCCCAGAGAGGCCAAAGTACAGTGGAAGGTGGATAACGC,Kappa,IGKV1-5*03,GACATCCAGATGACCCAGTCTCCTTCCACCTTGTCTGCATCTGTAGGAGACAGAGTCACCATCACTTGCCGGGCCAGTCACAGTATTAATACTTGGTTGGCCTGGTATCAGCAGAAACCAGGGAAAGCCCCTAAACTCCTGATATATAAGGCGTCTTCTTTAGAAAAGGGGGTCCCATCAAGGTTCAGCGGCAGTGGATCTGGGACCGAATTCACTCTCACCATCAACAGCCTGCAGCCTGATGATTTTGCAACTTATTACTGCCAACACTATAATAGTTATCCATTCGCTTTTGGCCAGGGGACCATGCTGGAGATCGAAC,DIQMTQSPSTLSASVGDRVTITCRASHSINTWLAWYQQKPGKAPKLLIYKASSLEKGVPSRFSGSGSGTEFTLTINSLQPDDFATYYCQHYNSYPFAFGQGTMLEIE,DIQMTQSPSTLSASVGDRVTITCRASQSISSWLAWYQQKPGKAPKLLIYKASSLESGVPSRFSGSGSGTEFTLTISSLQPDDFATYYCQQYNSYXXXFGQGTKLEIK,QHYNSYPFA,QLQVQESGPGLVKPSETLSLTCTVSGASSSIKKYYWGWIRQSPGKGLEWIGSIYSSGSTQYNPALGSRVTLSVDTSQTQFSLRLTSVTAADTATYFCARQGADCTDGSCYLNDAFDVWGRGTVVTVSS[SEP]DIQMTQSPSTLSASVGDRVTITCRASHSINTWLAWYQQKPGKAPKLLIYKASSLEKGVPSRFSGSGSGTEFTLTINSLQPDDFATYYCQHYNSYPFAFGQGTMLEIE,Unsorted-B-Cells,None,human,390c,"James et al, 2020",65 to 70,DIQMTQSPSTLSASVGDRVTITCRASHSINTWLAWYQQKPGKAPKLLIYKASSLEKGVPSRFSGSGSGTEFTLTINSLQPDDFATYYCQHYNSYPFAFGQGTMLEIE,DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNNKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSSPQTFGQGTKLEIK,QLQVQESGPGLVKPSETLSLTCTVSGASSSIKKYYWGWIRQSPGKGLEWIGSIYSSGSTQYNPALGSRVTLSVDTSQTQFSLRLTSVTAADTATYFCARQGADCTDGSCYLNDAFDVWGRGTVVTVSS,349,63.7168,2.39702,345,63.7168,IGHV4,IGKV1
2,TGGGAGGCTTTCTAAGAGTCATGGATCTCATGTGCAAGAAAATGAAGCACCTGTGGTTCTTCCTCCTGCTGGTGGCGGCTC

In [28]:
using DataFrames, PyCall

# Import Python modules
sns = pyimport("seaborn")
plt = pyimport("matplotlib.pyplot")
pd = pyimport("pandas")

# Assume your Julia DataFrame is named `df`
py_df_merged_final = pd.DataFrame(Dict(col => df_merged_final_filtered[!, col] for col in names(df_merged_final_filtered)));

In [29]:
#palette = {"Kappa": "orange", "Lambda": "orangered"}

# Create a palette dictionary mapping groups to colors:
palette = Dict("Kappa" => "lightskyblue", "Lambda" => "blue")


# Start a fresh figure
plt.figure()

bins = 40
sns.histplot(
    data=py_df_merged_final,
    x="calculated_similarity",
    hue="locus_light",
    bins=bins,
    kde=true,
    palette=palette,
    edgecolor="white",
    alpha=0.3
)

plt.xlabel("Similarity betw. generated and true light sequence")
plt.ylabel("Frequency")
ax = plt.gca()
ax.get_legend().set_title("Locus")

# Save before showing the plot
plt.savefig("/ibmm_data2/oas_database/paired_lea_tmp/paired_model/peak_analysis/bert2gpt/plots/control/locus_light-3_histplot_bins_40_poly_kde_blue.png", dpi=300)
plt.show()

In [30]:
# remove all rows which have L in the column 
df_merged_final_filtered = filter(row -> row.locus_light != "Lambda", df_merged_final_filtered)

# remove all rows which have D in the string in the column v_gene_light_family
#df_merged_final_filtered = filter(row -> !occursin("D", row.v_gene_light_family), df_merged_final_filtered)

Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age,sequence_alignment_aa_light_1,generated_sequence_light,input_heavy_sequence,BLOSUM_score,similarity,perplexity,calculated_blosum,calculated_similarity,v_gene_heavy_family,v_gene_light_family
,String,String1,String15,String,String,String,String,String,String,String15,String,String,String,String31,String,String31,String7,String7,String15,String31,String15,String,String,String,Int64,Float64,Float64,Int64,Float64,String,String
1,GGGAGGCTTTCTAAGAGTCATGGATCTCATGTGCAAGAAAATGAAGCACCTGTGGTTCTTCCTCCTGCTGGTGGCGGCTCCCAGATGGATCGTGTCCCAGTTGCAGGTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACGTGCACTGTCTCTGGTGCCTCCAGTAGTATTAAAAAATACTACTGGGGCTGGATCCGCCAGTCCCCCGGGAAGGGTCTGGAGTGGATTGGGAGTATCTATTCTAGCGGGAGTACGCAGTACAATCCGGCCCTCGGGAGTCGAGTCACCCTCTCCGTGGACACGTCCCAGACCCAGTTCTCCCTGAGGCTGACCTCTGTGACCGCCGCAGACACGGCTACGTACTTCTGTGCGAGACAGGGGGCGGATTGTACTGACGGCAGCTGCTACCTCAATGATGCTTTTGATGTTTGGGGCCGCGGGACAGTGGTCACCGTCTCTTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCGACAGCACCCCCCAAGATGGGAACGTGGTCGTCGCATGCCTGGTCCAGGGCTTCTTCCCCCAGGAGCCACTCAGTGTGACCTGGAGCGAAAGCGGACAGAACGTGACCGCCAGAAACTTCCC,H,IGHV4-39*01,CAGTTGCAGGTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACGTGCACTGTCTCTGGTGCCTCCAGTAGTATTAAAAAATACTACTGGGGCTGGATCCGCCAGTCCCCCGGGAAGGGTCTGGAGTGGATTGGGAGTATCTATTCTAGCGGGAGTACGCAGTACAATCCGGCCCTCGGGAGTCGAGTCACCCTCTCCGTGGACACGTCCCAGACCCAGTTCTCCCTGAGGCTGACCTCTGTGACCGCCGCAGACACGGCTACGTACTTCTGTGCGAGACAGGGGGCGGATTGTACTGACGGCAGCTGCTACCTCAATGATGCTTTTGATGTTTGGGGCCGCGGGACAGTGGTCACCGTCTCTTCAG,QLQVQESGPGLVKPSETLSLTCTVSGASSSIKKYYWGWIRQSPGKGLEWIGSIYSSGSTQYNPALGSRVTLSVDTSQTQFSLRLTSVTAADTATYFCARQGADCTDGSCYLNDAFDVWGRGTVVTVSS,QLQLQESGPGLVKPSETLSLTCTVSGGSISSSSYYWGWIRQPPGKGLEWIGSIYYSGSTYYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARXXXXCSGGSCYXXDAFDVWGQGTMVTVSS,ARQGADCTDGSCYLNDAFDV,AGGAGTCAGACCCAGTCAGGACACAGCATGGACATGAGGGTCCCCGCTCAGCTCCTGGGGCTCCTGCTGCTCTGGCTCCCAGGTGCCAAATGTGACATCCAGATGACCCAGTCTCCTTCCACCTTGTCTGCATCTGTAGGAGACAGAGTCACCATCACTTGCCGGGCCAGTCACAGTATTAATACTTGGTTGGCCTGGTATCAGCAGAAACCAGGGAAAGCCCCTAAACTCCTGATATATAAGGCGTCTTCTTTAGAAAAGGGGGTCCCATCAAGGTTCAGCGGCAGTGGATCTGGGACCGAATTCACTCTCACCATCAACAGCCTGCAGCCTGATGATTTTGCAACTTATTACTGCCAACACTATAATAGTTATCCATTCGCTTTTGGCCAGGGGACCATGCTGGAGATCGAACGAACTGTGGCTGCACCATCTGTCTTCATCTTCCCGCCATCTGATGAGCAGTTGAAATCTGGAACTGCCTCTGTTGTGTGCCTGCTGAATAACTTCTATCCCAGAGAGGCCAAAGTACAGTGGAAGGTGGATAACGC,Kappa,IGKV1-5*03,GACATCCAGATGACCCAGTCTCCTTCCACCTTGTCTGCATCTGTAGGAGACAGAGTCACCATCACTTGCCGGGCCAGTCACAGTATTAATACTTGGTTGGCCTGGTATCAGCAGAAACCAGGGAAAGCCCCTAAACTCCTGATATATAAGGCGTCTTCTTTAGAAAAGGGGGTCCCATCAAGGTTCAGCGGCAGTGGATCTGGGACCGAATTCACTCTCACCATCAACAGCCTGCAGCCTGATGATTTTGCAACTTATTACTGCCAACACTATAATAGTTATCCATTCGCTTTTGGCCAGGGGACCATGCTGGAGATCGAAC,DIQMTQSPSTLSASVGDRVTITCRASHSINTWLAWYQQKPGKAPKLLIYKASSLEKGVPSRFSGSGSGTEFTLTINSLQPDDFATYYCQHYNSYPFAFGQGTMLEIE,DIQMTQSPSTLSASVGDRVTITCRASQSISSWLAWYQQKPGKAPKLLIYKASSLESGVPSRFSGSGSGTEFTLTISSLQPDDFATYYCQQYNSYXXXFGQGTKLEIK,QHYNSYPFA,QLQVQESGPGLVKPSETLSLTCTVSGASSSIKKYYWGWIRQSPGKGLEWIGSIYSSGSTQYNPALGSRVTLSVDTSQTQFSLRLTSVTAADTATYFCARQGADCTDGSCYLNDAFDVWGRGTVVTVSS[SEP]DIQMTQSPSTLSASVGDRVTITCRASHSINTWLAWYQQKPGKAPKLLIYKASSLEKGVPSRFSGSGSGTEFTLTINSLQPDDFATYYCQHYNSYPFAFGQGTMLEIE,Unsorted-B-Cells,None,human,390c,"James et al, 2020",65 to 70,DIQMTQSPSTLSASVGDRVTITCRASHSINTWLAWYQQKPGKAPKLLIYKASSLEKGVPSRFSGSGSGTEFTLTINSLQPDDFATYYCQHYNSYPFAFGQGTMLEIE,DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNNKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSSPQTFGQGTKLEIK,QLQVQESGPGLVKPSETLSLTCTVSGASSSIKKYYWGWIRQSPGKGLEWIGSIYSSGSTQYNPALGSRVTLSVDTSQTQFSLRLTSVTAADTATYFCARQGADCTDGSCYLNDAFDVWGRGTVVTVSS,349,63.7168,2.39702,345,63.7168,IGHV4,IGKV1
2,TGGGAGGCTTTCTAAGAGTCATGGATCTCATGTGCAAGAAAATGAAGCACCTGTGGTTCTTCCTCCTGCTGGTGGCGGCTC

In [32]:
#palette = {"Kappa": "orange", "Lambda": "orangered"}

# Create a palette dictionary mapping groups to colors:
#palette = Dict("Kappa" => "lightskyblue", "Lambda" => "blue")

py_df_merged_final_filtered = pd.DataFrame(Dict(col => df_merged_final_filtered[!, col] for col in names(df_merged_final_filtered)))


palette = sns.color_palette("tab10", 10)


# Start a fresh figure
plt.figure()

bins = 40
sns.histplot(
    data=py_df_merged_final,
    x="calculated_similarity",
    hue="v_gene_light_family",
    bins=bins,
    kde=true,
    palette=palette,
    edgecolor="white",
    alpha=0.3
)

plt.xlabel("Similarity betw. generated and true light sequence")
plt.ylabel("Frequency")
ax = plt.gca()
ax.get_legend().set_title("V gene family")

# Save before showing the plot
plt.savefig("/ibmm_data2/oas_database/paired_lea_tmp/paired_model/peak_analysis/bert2gpt/plots/control/v_gene_histplot_bins_40_poly_kde_blue.png", dpi=300)
plt.show()